In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time, requests, os, xlrd, sys
from datetime import timedelta,date
pd.set_option('precision',3)

def calc_state(data):
    if data < 10.0:
        return u'极度低估'
    elif 10 <= data  and data < 20:
        return u'低估'
    elif 20 <= data  and data < 40:
        return u'正常偏低'
    elif 40 <= data  and data < 60:
        return u'正常'
    elif 60 <= data  and data < 80:
        return u'正常偏高'
    elif 80 <= data  and data < 90:
        return u'高估'
    elif 90 <= data:
        return u'极度高估'

def convert_code(code):
    if code.endswith('XSHG'):
        return 'sh' + code[0:6]
    elif code.endswith('XSHE'):
        return 'sz' + code[0:6]    

def get_index_pe_pb_date(code,date):
    '''指定日期的指数PE_PB（等权重）'''
    stocks = get_index_stocks(code, date)
    q = query(valuation).filter(valuation.code.in_(stocks))
    df = get_fundamentals(q, date)
    if len(df)>0:
        pe = len(df)/sum([1/p if p>0 else 0 for p in df.pe_ratio])
        pb = len(df)/sum([1/p if p>0 else 0 for p in df.pb_ratio])
        return (round(pe,2), round(pb,2))
    else:
        return float('NaN')

def get_index_pe_pb(code, start_date=None, end_date=None):
    '''指数历史PE_PB'''
    if start_date is None:
        start_date = get_security_info(code).start_date
        if start_date < date(2005,01,04): #只计算2005年以来的数据
            start_date = date(2005,01,04)
    if end_date is None:
        end_date = pd.datetime.today()  - timedelta(1)     #如果有误，请删除#号 ，获取的是前一天的数据
    x = get_price(code, start_date=start_date, end_date=end_date, frequency='daily', fields='close')
    date_list = x.index.tolist()
#     print date_list
    pe_list = []
    pb_list = []
    for d in date_list: #交易日
        pe_pb = get_index_pe_pb_date(code,d)
        pe_list.append(pe_pb[0])
        pb_list.append(pe_pb[1])
    df = pd.DataFrame({'PE': pd.Series(pe_list, index=date_list),
                        'PB': pd.Series(pb_list, index=date_list)})
 
    return df

def get_hk_data(data_root='./'):
    '''获取港股数据'''
    for idx in ['hsi', 'hscei']:    
        url = '''http://sc.hangseng.com/gb/www.hsi.com.hk/HSI-Net/static/revamp/contents/\
en/dl_centre/reports_stat/monthly/pe/%s.xls''' % (idx)
        data_path = '%s%s.xls' % (data_root,idx)
        ret = requests.get(url)
        if ret.ok is True:
            with open(data_path, "wb") as f:
                f.write(ret.content)
            print 'write data: %s' % (data_path)
    
        for i in range(10):
            day_str = (date.today() - timedelta(1) - timedelta(i)).strftime('%d%b%y')
            url = 'http://sc.hangseng.com/gb/www.hsi.com.hk/HSI-Net/static/revamp/contents/en/indexes\
/report/%s/idx_%s.csv' % (idx, day_str[1:] if day_str.startswith('0') else day_str)
            print url
            ret = requests.get(url)
            if ret.ok is True:
                data_path = '%s_daily.csv' % (idx)
                with open(data_path, "wb") as f:                    
                    f.write(ret.content[2:].decode("utf16").encode('utf8'))
                    print 'write data: %s' % (data_path)
                break
                
def read_hk_data(idx, daily=True,data_root='./'):
    '''读取港股数据'''
    data_path = '%s%s.xls' % (data_root,idx)
    xlr_f = xlrd.open_workbook(data_path)
    table = xlr_f.sheet_by_index(0)
    day_idx = table.col_values(0)[13:]
    pe = table.col_values(1)[13:]
    pe = [item for item in pe if item is not u'']
    day_idx = [xlrd.xldate.xldate_as_datetime(
        item, datemode=0) for item in day_idx if item is not u'']
    df = pd.DataFrame(pe, index=day_idx, columns=['PE'])
    if daily is True:
        if idx == 'hscei':
            hscei_data = file('hscei_daily.csv', 'r').read().splitlines()[-1].replace('\"','').split('\t') #PE:[-1]
            df = df.append(pd.DataFrame(float(hscei_data[-1]),index=[pd.Timestamp(hscei_data[0])],columns=['PE']))
        elif idx == 'hsi':
            hsi_data = file('hsi_daily.csv', 'r').read().splitlines()[2].replace('\"','').split('\t') #PE:[9]
            df = df.append(pd.DataFrame(float(hsi_data[9]),index=[pd.Timestamp(hsi_data[0])],columns=['PE']))
    return df[df.iloc[-1].name.date()-timedelta(365 * 15) : ] # 只用最近十五年数据

def pe_pb_analysis(index_list=['000300.XSHG','000905.XSHG'],data_root='./'):
    '''PE_PB分析'''
    all_index = get_all_securities(['index'])
    hk_idx_name = {'hscei':u'国企指数','hsi':u'恒生指数'}
    pe_results = []
    pe_code_list = []
    pb_results = []
    pb_code_list = []
    #沪深
    for code in index_list:
        data_path = '%s%s_pe_pb.csv'%(data_root,convert_code(code))
        index_name = all_index.ix[code].display_name
        df_pe_pb = pd.DataFrame.from_csv(data_path)
        df_pe_pb = df_pe_pb[df_pe_pb.iloc[-1].name.date() - timedelta(365*10):] #最长十年的数据
        if len(df_pe_pb)<250*3: #每年250个交易日,小于3年不具有参考价值
#                 print code, 'samples:', len(df_pe_pb), index_name
            continue
        pe_ratio = len(df_pe_pb.PE[df_pe_pb.PE<df_pe_pb.iloc[-1].PE])/float(len(df_pe_pb.PE))*100
        pb_ratio = len(df_pe_pb.PB[df_pe_pb.PB<df_pe_pb.iloc[-1].PB])/float(len(df_pe_pb.PB))*100        
        pe_results.append([index_name, df_pe_pb.iloc[-1].PE, '%.2f'%pe_ratio, calc_state(pe_ratio),
                           min(df_pe_pb.PE), max(df_pe_pb.PE), '%.2f'%median(df_pe_pb.PE), '%.2f'%std(df_pe_pb.PE),
                           df_pe_pb.iloc[0].name.date()])
        pb_results.append([index_name, df_pe_pb.iloc[-1].PB, '%.2f'%pb_ratio, calc_state(pb_ratio),
                           min(df_pe_pb.PB), max(df_pe_pb.PB), '%.2f'%median(df_pe_pb.PB),'%.2f'%std(df_pe_pb.PB),
                           df_pe_pb.iloc[0].name.date()])
        pe_code_list.append(code)
        pb_code_list.append(code)
    #港股
##    for code in ['hsi', 'hscei']:
##        df_pe = read_hk_data(code)
##        pe_ratio = len(df_pe.PE[df_pe.PE<df_pe.iloc[-1].PE])/float(len(df_pe.PE))*100
##        pe_results.append([hk_idx_name[code], df_pe.iloc[-1].PE, '%.2f'%pe_ratio, calc_state(pe_ratio), 
##                           min(df_pe.PE), max(df_pe.PE),'%.2f'%median(df_pe.PE),'%.2f'%std(df_pe.PE), 
##                           df_pe.iloc[0].name.date()])
##        pe_code_list.append(code.upper())

#     print '估值日期: ', df_pe_pb.iloc[-1].name.date()
    date_str = df_pe_pb.iloc[-1].name.date().strftime('%Y%m%d')
    pe_columns=[u'名称', u'当前PE', u'百分位(%)', u'估值状态', u'最小', u'最大', u'中位数', u'标准差', u'起始日期']
    pe_df = pd.DataFrame(data=pe_results,index=pe_code_list,columns=pe_columns)
    pe_df.index = pe_df[u'名称']
    del pe_df[u'名称']
    pe_df.index.name = date_str
    
    pb_columns=[u'名称', u'当前PB', u'百分位(%)', u'估值状态', u'最小', u'最大', u'中位数', u'标准差', u'起始日期']
    pb_df = pd.DataFrame(data=pb_results,index=pb_code_list,columns=pb_columns)
    pb_df.index = pb_df[u'名称']
    del pb_df[u'名称']
    pb_df.index.name = date_str
    pe_df = pe_df.convert_objects(convert_numeric=True) #转换为数字型
    pb_df = pb_df.convert_objects(convert_numeric=True) #转换为数字型
    return (pe_df.sort([u'百分位(%)'],ascending=True), pb_df.sort([u'百分位(%)'],ascending=True))

def get_hs_data(index_list,data_root='./'):
    '''增量更新沪深指数估值数据'''
    for code in index_list:
        print u'正在计算:', code
        data_path = '%s%s_pe_pb.csv'%(data_root,convert_code(code))
        if os.path.exists(data_path):#增量更新
            df_pe_pb = pd.DataFrame.from_csv(data_path)
            start_date = df_pe_pb.iloc[-1].name + timedelta(1)
            df_pe_pb = pd.concat([df_pe_pb, get_index_pe_pb(code, start_date)]) 
        else:#初次计算
            print 'init'
            df_pe_pb = get_index_pe_pb(code)
        df_pe_pb.to_csv(data_path)
        df_pe_pb
#       df_pe_pb.plot(secondary_y=['PB'],figsize=(14,8),title=get_all_securities(['index']).ix[code].display_name,style=['k-.'])   #画图 可以删除

index_list =[    
#     '000016.XSHG','000300.XSHG','000905.XSHG','000852.XSHG','399006.XSHE','399005.XSHE',#宽指数
#     '000001.XSHG','399001.XSHE','000902.XSHG','000985.XSHG',#大盘指数
#     '000015.XSHG','000922.XSHG','000827.XSHG','000978.XSHG',#策略指数
    '000827.XSHG','000990.XSHG','000991.XSHG','000016.XSHG','000300.XSHG','000905.XSHG','000922.XSHG','000015.XSHG',
    #中证环保，全指医药消费，上证50，沪深300，中证500，中证红利，红利指数
    '000985.XSHG','000932.XSHG',#中证全指，中证消费
    '399967.XSHE','399006.XSHE','399975.XSHE','399550.XSHE','399812.XSHE','399971.XSHE',#中证军工、创业板指，证券，央视50，养老产业，中证传媒
]

#get_hk_data() #港股数据
get_hs_data(index_list) #沪深数据

(pe_df, pb_df) = pe_pb_analysis(index_list)
print pe_df.index.name



 
#for code in ['399975.XSHE','399971.XSHE',]:
#    data_path = '%s%s_pe_pb.csv'%('./',convert_code(code))
#    df_pe_pb = pd.DataFrame.from_csv(data_path)
#    q_pes = [df_pe_pb['PE'].quantile(i/10.0)  for i in range(11)]
#    q_pbs = [df_pe_pb['PB'].quantile(i/10.0)  for i in range(11)] 
#    df_pe_pb['10% PE']=q_pes[1]
#    df_pe_pb['50% PE']=q_pes[5]
#    df_pe_pb['90% PE']=q_pes[9]
#    df_pe_pb['10% PB']=q_pbs[1]
#    df_pe_pb['50% PB']=q_pbs[5]
#    df_pe_pb['90% PB']=q_pbs[9]
#    df_pe_pb.plot(secondary_y=['PB','10% PB','50% PB','90% PB'],figsize=(14,8),title=get_all_securities(['index']).ix[code].display_name,style=['k-.', 'k', 'g', 'y', 'r', 'g-.', 'y-.', 'r-.'])

pe_df.index.name = None

In [10]:
class Fraction(object):
    def __init__(self, top, bottom):
        self.num = top 
        self.den = bottom
        
    def __str__(self):
        return str(self.num) + "/" + str(self.den)

    def show(self):
        print(self.num, "/", self.den)
        
    def __add__(self, otherfraction):
        newnum = self.num * otherfraction.den + \
        self.den * otherfraction.num 
        newden = self.den * otherfraction.den
        common = self.gcd(newnum, newden)
        return Fraction(newnum//common, newden//common)

    @staticmethod
    def gcd(num, den):
        while num%den != 0:
            oldm = num 
            oldn = den
            
            num = oldn
            den = oldm % oldn 
        return den

    def __eq__(self, other):
        firstnum = self.num * other.den 
        secondnum = other.num * self.den
        return firstnum == secondnum


f1 = Fraction(11, 7)
f1.show()
f2 = Fraction(3, 7)
f2.show()

print(f1, f2, f1+f2)
print(f1 == f2)

11 / 7
3 / 7
11/7 3/7 2/1
False
